In [3]:
import math, random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
np.set_printoptions(precision=8)

seed = 42
random.seed(seed)
np.random.seed(seed)

%matplotlib inline

In [4]:
import re
import requests
from bs4 import BeautifulSoup

In [5]:
target_url = "http://travel.rakuten.co.jp/yado/kanagawa/hakone.html"

In [8]:
text_list = []

soup_plans = BeautifulSoup(requests.get(target_url).text, 'html.parser')
num_plan_pages = int(soup_plans.select('.pagingTitle')[0].select('em')[0].text)

for i in range(int(num_plan_pages / 30) + 1):
    url = target_url.replace('travel.rakuten.co.jp', 'search.travel.rakuten.co.jp/ds')
    url = re.sub(r'.html', ('-p%s' % (i+1)), url)
    print(f'Number of plan pages: {i+1:d}/{num_plan_pages:d} --> {url}')
    soup_plan_page = BeautifulSoup(requests.get(url).text, 'html.parser')

    for tags in soup_plan_page.select('.htlList .info'):
        tag = tags.select('h1 a')
        url = tag[0].get('href')
        url = re.sub(r'/[0-9]+.html.*$', '/review.html', url)
        soup_reviews = BeautifulSoup(requests.get(url).text, 'html.parser')
        num_review_pages = int(soup_reviews.select('.pagingTitle')[0].select('em')[0].text)

        for j in range(int(num_review_pages / 20) + 1):
            url = soup_reviews.select('.pagingNumber a')[0].get('href')
            url = re.sub(r'f_next=.*', 'f_next=', url)
            url = '{:}{:}'.format(url, (j * 20))
            print(f'Number of review pages: {j+1:d}/{num_review_pages:d} --> {url}')
            soup_review_page = BeautifulSoup(requests.get(url).text, 'html.parser')            

            for tags in soup_review_page.select('.commentBox'):
                tag = tags.select('.commentTitle a')[0]
                url = tag.get('href')
                soup_comments = BeautifulSoup(requests.get(url).text, 'html.parser')
                comment = soup_comments.select('.commentBox')[0]
                text = comment.select('.commentSentence')[0].get_text(strip=True)
                text = re.sub('\r\n', ' ', text)
                text = re.sub('\n',   ' ', text)
                text = re.sub('\t',   ' ', text)
                text_list.append(text)

            # 今回は練習用として1ページ目のレビューだけで中断する
            break

        # 今回は練習用として1プランだけで中断する
        break

    # 今回は練習用として1ページ目のプランだけで中断する
    break

Number of plan pages: 1/313 --> http://search.travel.rakuten.co.jp/ds/yado/kanagawa/hakone-p1
Number of review pages: 1/191 --> https://review.travel.rakuten.co.jp/hotel/voice/146841/?f_time=&f_keyword=&f_age=0&f_sex=0&f_mem1=0&f_mem2=0&f_mem3=0&f_mem4=0&f_mem5=0&f_teikei=&f_version=2&f_static=1&f_point=0&f_sort=0&f_next=0


In [9]:
text_df = pd.DataFrame(text_list)
text_df.columns = ['text']
print(text_df.shape)
display(text_df)

(20, 1)


,text
0,お料理が全部美味しかったです！お部屋も良かったですし、露天風呂も気持ち良かったです。また行こ...
1,失敗でした。この宿は県民割に参加していないので、楽天から予約しました。3000円しか補助され...
2,部屋付き露天風呂が良かった。
3,6月22日結婚60周年記念に夫婦で宿泊させて頂きました。洋室の部屋も露天風呂も広々として、更...
4,結婚式翌日に一泊宿泊しました。結構山道を登った先にあるので、場所は少し分かりづらいです。到着...
5,皆さんの口コミの通り、露天風呂は広かったです。 ただ、サイトに載っている写真と露天風呂の形状...
6,カップルで宿泊しましたが専用露天風呂が思っていた以上に広々としていて最高でした！畳に一部シミ...
7,妊娠の為事前にお伝えし、お料理や飲み物を色々変更して頂き本当にありがとうございました。お料理...
8,6/11（土）家内と二人で利用させていただきました。11年間の単身赴任期間に子達の世話も含め...
9,母娘旅行で宿泊させて頂きました。事前に妊婦と伝えていた為お料理や飲み物など変更してくださって...


### Ginza を使ってみる

In [11]:
!pip install ja-ginza > /dev/null 2>&1

In [13]:
import spacy
import ginza

# ja-ginza モデルのロード
nlp = spacy.load('ja_ginza')

# 単語連結モードを選択
ginza.set_split_mode(nlp, "C")

for p in nlp.pipeline:
    print(p)

('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7f312f520ad0>)
('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7f313c8eeb50>)
('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7f313c8ee850>)
('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x7f312f522360>)
('compound_splitter', <ginza.compound_splitter.CompoundSplitter object at 0x7f312ced2990>)
('bunsetu_recognizer', <ginza.bunsetu_recognizer.BunsetuRecognizer object at 0x7f312cf39890>)


### 文の解析 (形態素解析)

In [29]:
# ja-ginza モデルのロード
nlp = spacy.load('ja_ginza')

result_list = []

# 文の解析
for i, text in enumerate(text_list):
    doc = nlp(text)
    for j, sent in enumerate(doc.sents):
      for token in sent:
        result_list = result_list + [[i, j, token.i, token.text, token.lemma_, token.pos_, token.tag_]]
    j = 0

result_df = pd.DataFrame(result_list, columns = ['doc_no', 'sent_no', 'token_no', 'text', 'lemma', 'pos', 'tag'])      
result_df

,doc_no,sent_no,token_no,text,lemma,pos,tag
0,0,0,0,お,お,NOUN,接頭辞
1,0,0,1,料理,料理,NOUN,名詞-普通名詞-サ変可能
2,0,0,2,が,が,ADP,助詞-格助詞
3,0,0,3,全部,全部,ADV,名詞-普通名詞-副詞可能
4,0,0,4,美味しかっ,美味しい,ADJ,形容詞-一般
...,...,...,...,...,...,...,...
2065,19,4,101,て,て,SCONJ,助詞-接続助詞
2066,19,4,102,み,みる,VERB,動詞-非自立可能
2067,19,4,103,たい,たい,AUX,助動詞
2068,19,4,104,です,です,AUX,助動詞


### 係り受け解析

In [33]:
from spacy import displacy

# 文の解析
for i, text in enumerate(text_list):
    doc = nlp(text)
    for j, sent in enumerate(doc.sents):
        print(f"文: {sent}")

        # 係り受け解析
        displacy.render(sent, style="dep", options={"compact":True}, jupyter=True)

        # 今回は練習用として1文だけで中断する
        break

    # 今回は練習用として1件だけで中断する
    break

原文: お料理が全部美味しかったです！


### 固有表現抽出

In [41]:
# 警告を抑止する
import warnings
warnings.simplefilter('ignore', UserWarning)

# 文の解析
for i, text in enumerate(text_list):
    doc = nlp(text)

    # 固有表現抽出
    displacy.render(doc, style="ent", jupyter=True)

    # 今回は練習用として5件だけで中断する
    if i >= 4:
      break

warnings.resetwarnings()